In [ ]:
import pandas as pd
from environments.forex_tradning import tgym

In [ ]:
parameters = {
"observation_list":["_dt","open","high","low","close","minute","hour","weekday","week","month","year","RSI","SMA"],
}
parameters = {
    "observation_list":["open","high","low","close","minute","hour","weekday","RSI","SMA"],
    "stop_loss_max":200,
    "profit_taken_max":400    
}
file = "./data/split/GBPUSD/weekly/GBPUSD_20191110_W_148.csv"
file = "./data/split/GBPUSD/weekly/GBPUSD_20191208_W_1523.csvx"
df = pd.read_csv(file)
t = tgym(df, **parameters)

In [5]:
import time
import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# if model: del model # remove to demonstrate saving and loading
model_name='./data/models/GBPUSD-week-20211021135753'
model = PPO.load(model_name)

start_time = time.time()
obs = t.reset()
t.current_step=0
done = False
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = t.step(action)
    t.render(mode='human')
print(f"--- running time: {(time.time() - start_time)}---")

  TicketSymbol  Type              ActionTime                                 ActionPrice   MaxDD              CloseTimeClosePrice                                     Reward  SL      PT      DateDuration        Status  

       2 ABCD        0 2/12/2019 0:05                       1.10572      -14 2/12/2019 14:40  1.10772                      190  200  200                    2        1
       4 ABCD        0 2/12/2019 0:10                       1.10564       -4 2/12/2019 14:40  1.10764                      190  200  200                    2        1
       6 ABCD        0 2/12/2019 0:15                       1.10566       -7 2/12/2019 14:40  1.10766                      190  200  200                    2        1
      10 ABCD        0 2/12/2019 0:25                       1.10569       -9 2/12/2019 14:40  1.10769                      190  200  200                    2        1

      14 GBPUSD      0 2/12/2019 14:50                      1.10163       -1 2/12/2019 15:55  1.10363          

In [ ]:
import time
import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Parallel environments

start_time = time.time()
model_name = "./data/models/GBPUSD-week-" + datetime.datetime.now().strftime('%Y%m%d%H%M%S')
print(f'save model as {model_name}')
model = PPO("MlpPolicy", t, verbose=1)
model.learn(total_timesteps=250000)
model.save(model_name)
print(f"--- trainning time: {(time.time() - start_time)}---")


In [ ]:
0 > -22

In [ ]:
!python test.py >

In [ ]:
!script_output=$(python test.py 2>&1)
!echo $script_output